In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
import os
import matplotlib.pyplot as plt

In [22]:
#CRIANDO UMA LISTA QUE CONTÉM TODOS OS TICKER DOS PEERS DIRETOS DO GARE,PARA FAZER UM ACOMPANHAMENTO MAIS PROXIMO
PEERS_DIRETOS = ['KNRI11.SA', 'TVRI11.SA', 'ALZR11.SA', 'RBRP11.SA', 'SARE11.SA',
                 'HGRU11.SA', 'TRXF11.SA', 'RBVA11.SA', 'VIUR11.SA', 'HGLG11.SA',
                 'BTLG11.SA', 'XPLG11.SA', 'BRCO11.SA', 'LVBI11.SA', 'VILG11.SA', 'GARE11.SA']
TIJOLO = ['HIB Tijolo', 'XXXX', 'Renda Urbana', 'Logistico', 'Shopping', 'Lajes', 'Outros Imóveis']

                           Open  High   Low  Close  Volume  Dividends  \
Date                                                                    
2024-12-16 00:00:00-03:00  8.23  8.28  8.13   8.13  543796        0.0   
2024-12-17 00:00:00-03:00  8.22  8.22  8.00   8.08  816412        0.0   

                           Stock Splits  
Date                                     
2024-12-16 00:00:00-03:00           0.0  
2024-12-17 00:00:00-03:00           0.0  


In [ ]:
#PUXO A BASE DE ATIVOS BASE QUE EU USO PARA FAZER AS ANALISES 
df_ativos = pd.read_excel('Base_ativos.xlsx')

#CRIO UMA FUNÇÃO PARA EXCLUIR A PARTE DO TICKER ".SA"
def excluir_5_dig(COTA):
    return str(COTA)[:6]

#CRIO UMA FUNÇÃO PARA IDENTIFICAR OS PEERS DO GARE11 NA TABELA
def peers(x):
    if x in PEERS_DIRETOS:
        return 'PEER'
    else:
        return 'NAO PEER'
    
def tijolo_papel(x):
    if x == 'Recebivel':
        return "Papel"
    elif x == "XXXX":
        return "GARE11"
    else:
        return "Tijolo"


def pl_maior_500000000(patri):
    if patri > 500000000:
        return "MAIOR"
    else:
        return "MENOR"
    
def liq_maior_1MM(liq):
    if liq > 1000000:
        return "MAIOR"
    else:
        return "MENOR"

In [ ]:
df_ativos['PL>500MM'] = df_ativos["VP"].apply(pl_maior_500000000)
df_ativos['LIQ>1MM'] = df_ativos["LIQUIDEZ"].apply(liq_maior_1MM)
df_ativos

In [ ]:
df_TIJOLO = df_ativos[df_ativos['TIPO'].isin(TIJOLO) & (df_ativos['VP'] > 500000000)]

In [ ]:
tic = 'TICKER'
liq = 'LIQUIDEZ'
cotistas = 'COTISTAS'
pl = 'VP'
#CRIO UM DATAFRAME COM AS MAIORES QUEDAS DO DIA ENTRE OS PEERS DO GARE E O GARE
TOP10_LIQUIDEZ = df_TIJOLO[[tic, liq]]
TOP10_LIQUIDEZ['LIQUIDEZ'] = TOP10_LIQUIDEZ['LIQUIDEZ'].round(0)
TOP10_LIQUIDEZ = TOP10_LIQUIDEZ.sort_values(by=['LIQUIDEZ'], ascending=False).iloc[:10, :]
TOP10_LIQUIDEZ['TICKER'] = TOP10_LIQUIDEZ['TICKER'].apply(excluir_5_dig)
TOP10_LIQUIDEZ

In [ ]:
import matplotlib.pyplot as plt

# Dados fornecidos
data = TOP10_LIQUIDEZ

# Criar o DataFrame
df = pd.DataFrame(data)

# Definir as cores para destacar o GARE11.SA
cores = ['#91BDE7' if fundo != 'GARE11' else '#3B5279' for fundo in df['TICKER']]

# Criar o gráfico de barras
fig, ax = plt.subplots(figsize=(9, 4))
barras = ax.bar(df['TICKER'], df['LIQUIDEZ'], color=cores)

# Adicionar rótulos de barra com separadores de milhares (usando ponto)
for barra in barras:
    altura = barra.get_height()
    ax.text(
        barra.get_x() + barra.get_width() / 2, altura + 50000,  # Ajuste de posição vertical
        f'{altura:,.0f}'.replace(',', '.'),  # Formatar com separador de milhares como ponto
        ha='center', va='bottom', fontsize=10
    )

# Personalizar o gráfico
ax.set_xlabel('')
ax.set_title('', fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.get_yaxis().set_visible(False)

# Ajustar layout
plt.xticks(rotation=45)
plt.tight_layout()

fig.savefig('TOP10_LIQUIDEZ.png', transparent = True, dpi=300, bbox_inches='tight')

# Exibir o gráfico
plt.show()

In [ ]:
tic = 'TICKER'
cotistas = 'COTISTAS'
#CRIO UM DATAFRAME COM AS MAIORES QUEDAS DO DIA ENTRE OS PEERS DO GARE E O GARE
TOP10_cotistas = df_TIJOLO[[tic, cotistas]]
TOP10_cotistas = TOP10_cotistas.sort_values(by=['COTISTAS'], ascending=False).iloc[:10, :]
TOP10_cotistas['TICKER'] = TOP10_cotistas['TICKER'].apply(excluir_5_dig)
TOP10_cotistas

In [ ]:
# Dados fornecidos
data = TOP10_cotistas

# Criar o DataFrame
df = pd.DataFrame(data)

# Definir as cores para destacar o GARE11.SA
cores = ['#91BDE7' if fundo != 'GARE11' else '#3B5279' for fundo in df['TICKER']]

# Criar o gráfico de barras
fig, ax = plt.subplots(figsize=(9, 4))
barras = ax.bar(df['TICKER'], df['COTISTAS'], color=cores)

# Adicionar rótulos de barra com separadores de milhares (usando ponto)
for barra in barras:
    altura = barra.get_height()
    ax.text(
        barra.get_x() + barra.get_width() / 2, altura + 50000,  # Ajuste de posição vertical
        f'{altura:,.0f}'.replace(',', '.'),  # Formatar com separador de milhares como ponto
        ha='center', va='bottom', fontsize=10
    )

# Personalizar o gráfico
ax.set_xlabel('')
ax.set_title('', fontsize=14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.get_yaxis().set_visible(False)

# Ajustar layout
plt.xticks(rotation=45)
plt.tight_layout()

fig.savefig('TOP10_COTISTAS.png', transparent = True, dpi=300, bbox_inches='tight')

# Exibir o gráfico
plt.show()

In [ ]:
tic = 'TICKER'
liq = 'LIQUIDEZ'
pl = 'VP'
#CRIO UM DATAFRAME COM AS MAIORES QUEDAS DO DIA ENTRE OS PEERS DO GARE E O GARE
TOP10_LIQxPL = df_TIJOLO[[tic, liq, pl]]
TOP10_LIQxPL['VP'] = TOP10_LIQxPL['VP']/1000
TOP10_LIQxPL['LIQUIDEZ'] = TOP10_LIQxPL['LIQUIDEZ']/1000
TOP10_LIQxPL['LIQUIDEZ'] = TOP10_LIQxPL['LIQUIDEZ'].round(0)
TOP10_LIQxPL['VP'] = TOP10_LIQxPL['VP'].round(0)
TOP10_LIQxPL['LIQ/PL'] = TOP10_LIQxPL['LIQUIDEZ']/TOP10_LIQxPL['VP']
TOP10_LIQxPL = TOP10_LIQxPL.sort_values(by=['LIQ/PL'], ascending=False).iloc[:10, :]
TOP10_LIQxPL['TICKER'] = TOP10_LIQxPL['TICKER'].apply(excluir_5_dig)

In [ ]:
data = TOP10_LIQxPL

# Criar o DataFrame
df = pd.DataFrame(data)

cores = ['#3B5279' if ticker == 'GARE11' else '#91BDE7' for ticker in df['TICKER']]

# Criar o gráfico de barras
fig, ax = plt.subplots(figsize=(9, 4))
barras = ax.bar(df['TICKER'], df['LIQ/PL'], color=cores)

# Adicionar rótulos de barra como porcentagem
for barra in barras:
    altura = barra.get_height()
    ax.text(
        barra.get_x() + barra.get_width() / 2, altura + 0.0001,  # Ajuste de posição vertical
        f'{altura:.2%}',  # Formatar como porcentagem
        ha='center', va='bottom', fontsize=10
    )

# Personalizar o gráfico
ax.set_xlabel('')
ax.set_ylabel('')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.get_yaxis().set_visible(False)

# Ajustar layout
plt.xticks(rotation=45)
plt.tight_layout()

fig.savefig('TOP10_LIQxPL.png', transparent=True, dpi=300, bbox_inches = 'tight')

# Exibir o gráfico
plt.show()

In [ ]:
#CRIO VARIAVES PARA ARMAZENAR AS DATAS QUE PRECISO PEGAR OS PREÇOS DAS COTAS DOS FUNDOS
d_1 = datetime(2024, 12, 18, 17, 5, 0)
d_2 = datetime(2024, 12, 17, 17, 5, 0)
d_now = datetime.now()

#CRIO A COLUNA DE PREÇO COTA E DEIXO ELA VAZIA PARA ARMAZENAR COM OS PREÇOS DAS COTAS QUE SERÃO PUXADAS PELA FUNÇÃO
df_ativos['PREÇO COTA'] = None

#COLOCO A COLUNA 'TICKER' COMO O INDEX DA COLUMA E CRIO A FUNÇÃO PARA PASSAR POR TODOS OS DADOS PRESENTE NA COLUNA TICKER
for index, row in df_ativos.iterrows():
    #INSIRO OS TICKER PRESENTE NO DF DENTRO DA VARIAVEL TICKER, USANDO O FOR PARA CRIAR O LAÇO DE REPETIÇÃO
    ticker = row['TICKER']
    
    try:
        #USANDO O YAHOO FINANCE EU ARMAZENO OS TICKERS DENTRO DA VARIAVEL
        fundo = yf.Ticker(ticker)
        
        #BUSCO O HISTORICO DOS ULTIMOS DOIS DIAS (D0 E D-2)
        historico = fundo.history(start=d_2, end=d_1)  #INCLUINDO OS ULTIMOS DOIS DIAS
        
        #GARANTINDO QUE TENHO DADOS OS= SUFICIENTE PARA FAZER A ANALISE
        if len(historico) >= 1:
            #EXTRAINDO O PREÇO DE FECHAMENTO MAIS RECENTE DISPONIVEL
            preco_fechamento_ontem = historico['Close'].iloc[-1]
            
            #ARMAZENO O PREÇO DE FECHAMENTO NA COLUNA DE PREÇO COTA QUE ANTERIORMENTE ESTAVA VAZIA
            df_ativos.at[index, 'PREÇO COTA'] = round(preco_fechamento_ontem, 2)
        else:
            #SE NÃO POSSUI DADOS O SUFICIENTE O PREÇO COTA DO TICKER É ARMAZENADO COMO NaN
            df_ativos.at[index, 'PREÇO COTA'] = None
    
    except Exception as e:
        df_ativos.at[index, 'PREÇO COTA'] = f'Erro: {e}'


#-------------------------------------------------------FAZENDO O MESMO PROCESSO, MAS BUSCANDO O PREÇO DO DIA ANTERIOR AO BUSCADO ACIMA-------------------------------------------------------

#CRIO A COLUNA DE PREÇO COTA D_2 QUE VAI ARMAZENAR O PREÇO DA COTA DE FECHAMENTO DO PENULTIMO DIA
df_ativos['PREÇO COTA_D_2'] = None

#PASSO POR TODOS OS TICKERS E OS ARMAZENO DENTRO DA VARIAVEL ticker
for index, row in df_ativos.iterrows():
    ticker = row['TICKER']
    
    try:
        # OBTENHO O HISTÓRICO DO FUNDO PARA O ULTIMO DIA O ULTIMO DIA DE FECHAMENTO
        fundo = yf.Ticker(ticker)
        historico_2 = fundo.history(start = d_2, end = d_1)  

        #VERIFICAR SE A DADOS SUFICIENTES PARA ANALISAR A OPERAÇÃO
        if len(historico) >= 1:
            preco_fechamento_anteontem = historico_2['Close'].iloc[0]
            df_ativos.at[index, 'PREÇO COTA_D_2'] = round(preco_fechamento_anteontem, 2)
        else:
            df_ativos.at[index, 'PREÇO COTA_D_2'] = 0
    
    except Exception as e:
        df_ativos.at[index, 'PREÇO COTA_3'] = 'Erro'

In [ ]:
#CALCULANDO PREÇO DA COTA D-1, D-2 E VARIAÇÃO
GARE11=yf.Ticker('GARE11.SA')
print('Dia anterior {:%Y-%m-%d}, dois dias atras {:%Y-%m-%d}'.format(d_1, d_2))

#COTA DE D-1 ---> o preço que vai retornar dessa função é o preço que eu coloco como start
cota_d_1 = GARE11.history(start = d_1, end = d_now)['Close'].iloc[0]
#COTA DE D-2
cota_d_2 = GARE11.history(start = d_2, end=d_1)['Close'].iloc[-2]
#VARIAÇÃO
VAR = ((cota_d_1-cota_d_2)/cota_d_2)*100
#COTA DE D-1 MAXIMO
maximo_d_1 = GARE11.history(start = d_1, end = d_now)['High'].iloc[0]
#COTA DE D-1 MINIMO
minimo_d_1 = GARE11.history(start = d_1, end = d_now)['Low'].iloc[0]

print('DATA: {:%Y-%m-%d}: VALOR DA COTA {:.2f}'.format(d_1, cota_d_1))
print('DATA: {:%Y-%m-%d}: VALOR DA COTA {:.2f}'.format(d_2, cota_d_2))
print('DATA: {:%Y-%m-%d}: MAXIMO {:.2F}'. format(d_1, maximo_d_1))
print('DATA: {:%Y-%m-%d}: MINIMO {:.2F}'. format(d_1, minimo_d_1))
print('VARIAÇÃO: {:.2f}%'.format(VAR))

In [ ]:
data = {'Preço_D-1':[cota_d_1],
        'Preço_D-2':[cota_d_2],
        'Var':[VAR],
        'Maximo':[maximo_d_1],
        'Minimo':[minimo_d_1],}

tab_GARE11 = pd.DataFrame(data)

tab_GARE11

In [ ]:
df_ativos

In [ ]:
#GARANTO QUE A COLUNA 'PREÇO COTA' ESTÁ EM FLOAT
df_ativos['PREÇO COTA'] = df_ativos['PREÇO COTA'].astype(float)
#CRIO O COLUNA DE P/VP
df_ativos['P/VP'] = ((df_ativos['COTAS EMITIDAS']*df_ativos['PREÇO COTA'])/df_ativos['VP']).round(2)
#FAÇO MANIPULAÇÕES PARA COLOCAR AS COLUNAS DE DIVIDENDO COM DUAS CASAS DECIMAIS
df_ativos['DIV - Y1'] = df_ativos['DIV - Y1'].round(2)
df_ativos['DIV- Y3'] = df_ativos['DIV- Y3'].round(2)
df_ativos['DIV - LTM'] = df_ativos['DIV - LTM'].round(2)
#RETIRO AS COLUNAS INUTEIS
#df = df.drop(columns=['PREÇO COTA_3', 'NOME FUNDO'])
#CRIO UMA COPIA DO DATAFRAME PRA FAZER EVENTUAIS MANIPULAÇÕES
df_2 = df_ativos
#CRIO NOVAS COLUNAS DE VARIAÇÃO
df_2['VAR'] = (((df_2['PREÇO COTA'] - df_2['PREÇO COTA_D_2'])/df_2['PREÇO COTA_D_2'])*100).round(2)
df_2['VAR_ANO'] = ((df_2['PREÇO COTA']/df_2['COTA - 2024']-1)*100).round(2)
df_2['VAR_LTM'] = ((df_2['PREÇO COTA']/df_2['LTM']-1)*100).round(2)
df_2['VAR_MES'] = ((df_2['PREÇO COTA']/df_2['COTA NOV']-1)*100).round(2)
#REORGANIZO AS COLUNAS DO DATAFRAME
df_2 = df_2[['TICKER', 'TIPO', 'VP', 'COTAS EMITIDAS','P/VP', 'VAR', 'VAR_MES', 'VAR_ANO', 'VAR_LTM', 'PREÇO COTA', 'PREÇO COTA_D_2','LTM', 'COTA - 2024', 'COTA - 2023', 'COTA NOV',
         'DIV - Y1', 'DIV- Y3', 'DIV - LTM', 'IFIX']]
df_2['VP'] = (df_2['VP']/1000000).round(0)
df_2['VP'] = df_2['VP'].astype(int)
#CRIO AS COLUNAS DE DIVIDEND YIELD
df_2['DY - 1M'] = ((df_2['DIV - Y1']/df_2['PREÇO COTA'])*100).round(2)
df_2['DY - 3M'] = ((df_2['DIV- Y3']/df_2['PREÇO COTA'])*100).round(2)
df_2['DY - LTM'] = ((df_2['DIV - LTM']/df_2['PREÇO COTA'])*100).round(2)
#CRIO A COLUNA DE PEERS
df_2['PEERS'] = df['TICKER'].apply(peers)
#RETIRO COLUNAS QUE FICARAM INUTEIS
df_2 = df_2.drop(columns = ['COTA - 2024', 'COTA - 2023', 'COTA NOV', 'DIV - Y1', 'DIV- Y3', 'DIV - LTM'])
df_2 = df_2.rename(columns={'PREÇO COTA':'PRECO', 'PREÇO COTA_D_2':'PRECO_D2', 'DY - 1M':'DY-1M', 'DY - 3M':'DY-3M', 'DY - LTM':'DY-LTM'})

df_2['VAR'].astype(float)

df_2

In [ ]:
#CRIO UM DATAFRAME PARA ANALISAR APENAS OS PEERS DO GARE
df_3 = df_2[df_2['PEERS'] == 'PEER']
#EXCLUO COLUNAS INUTEIS
df_3 = df_3.rename(columns={'PREÇO COTA':'PRECO', 'PREÇO COTA_D_2':'PRECO_D2', 'DY - 1M':'DY-1M', 'DY - 3M':'DY-3M', 'DY - LTM':'DY-LTM'})
df_3 = df_3.drop(columns = ['VP', 'COTAS EMITIDAS', 'TIPO', 'PEERS', 'PRECO_D2', 'LTM'])
df_3 = df_3[['TICKER', 'PRECO', 'PL', 'P/VP', 'VAR', 'VAR_MES', 'VAR_ANO', 'VAR_LTM', 'DY-1M', 'DY-3M', 'DY-LTM', 'IFIX']]

df_3['TICKER'] = df_3['TICKER'].apply(excluir_5_dig)

df_3['VAR'] = df_3['VAR'].astype(float)

df_3['VAR'] = df_3['VAR'].round(2)

df_3

In [ ]:
df_4 = df_2[df_2['IFIX'] >= 0.0079]
df_4 = df_4.reset_index()
df_4['TIPO'] = df_4['TIPO'].apply(tijolo_papel)
df_4['TICKER'] = df_4['TICKER'].apply(excluir_5_dig)
df_4 = df_4.drop(columns = ['VP', 'COTAS EMITIDAS', 'PRECO_D2', 'LTM', 'PEERS'])
df_4 = df_4[['TICKER', 'TIPO', 'PL', 'PRECO', 'P/VP', 'VAR', 'VAR_MES', 'VAR_ANO', 'VAR_LTM', 'DY-1M', 'DY-3M', 'DY-LTM', 'IFIX']]
df_4

In [ ]:
#CRIO UM DATAFRAME PARA VER AS MEDIAS DO SETOR EM COMPARAÇÃO AO GARE
MEDIAS = df_2.groupby(by = 'TIPO')[['VAR', 'VAR_MES', 'VAR_ANO', 'VAR_LTM']].mean().round(2)
MEDIAS['VAR'] = MEDIAS['VAR'].astype(float)
MEDIAS['VAR'] = MEDIAS['VAR'].round(2)

MEDIAS

In [ ]:
tic = 'TICKER'
Preco = 'PRECO'
Var = 'VAR'
Var_ANO = 'VAR_ANO'
#CRIO UM DATAFRAME COM AS MAIORES QUEDAS DO DIA ENTRE OS PEERS DO GARE E O GARE
df_quedas = df_2[df_2['PRECO'] != 0]
TOP10_QUEDAS = df_quedas[[tic, Preco, Var]]
TOP10_QUEDAS = TOP10_QUEDAS.sort_values(by=['VAR'], ascending=True).iloc[:10, :]
TOP10_QUEDAS['VAR'] = TOP10_QUEDAS['VAR'].round(2)
TOP10_QUEDAS

In [ ]:
TOP10_ALTAS = df_2[[tic, Preco, Var]]
TOP10_ALTAS = TOP10_ALTAS.sort_values(by=['VAR'], ascending=False).iloc[:10, :]
TOP10_ALTAS

In [ ]:
TOP10_QUEDAS_ANO = df_quedas[[tic, Preco, Var_ANO]]
TOP10_QUEDAS_ANO = TOP10_QUEDAS_ANO.sort_values(by=['VAR_ANO'], ascending=True).iloc[:10, :]
TOP10_QUEDAS_ANO

In [ ]:
TOP10_ALTAS_ANO = df_2[[tic, Preco, Var_ANO]]
TOP10_ALTAS_ANO = TOP10_ALTAS_ANO.sort_values(by=['VAR_ANO'], ascending=False).iloc[:10, :]
TOP10_ALTAS_ANO

In [ ]:
#SALVANDO O DATAFRAME EM .XLSX EM UMA PASTA ESPECIFICA

#MAPEANDO O CAMINHO E SALVANDO A PLANILHA TRATADA COMO .XLSX
pasta_destino = r'C:\Users\LucasCavalcante\Desktop\ATIVO'
#CASO A PASTA NÃO EXISTA UMA SERA CRIADA IMEDIANTAMENT
os.makedirs(pasta_destino, exist_ok=True)
#INSERE EM UMA VARIAVEL O CAMINHO E O NOME DO ARQUIVO
caminho_arquivo_excel = os.path.join(pasta_destino, 'ativo_ifix.xlsx')
#EXPORTA O ARQUIVO PARA CSV E O SALVA NA PASTA QUE O CAMINHO INDICA
df.to_excel(caminho_arquivo_excel, index=False)

print('Arquivo salvo com sucesso')